Testing with ChatGPT

This Jupyter Notebook script contains an experimental usage of ChatGPT.

In [2]:
!pip install python-dotenv
!pip install openai
!pip install psycopg2 

In [3]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

Prompting Principles:
- Principle 1: Write clear and specific instructions
- Principle 2: Give the model time to "think"

In [5]:
data_dictionary = {
    "users": {
        "id": "unique identifier of each user",
        "first_name": "first name of each user",
        "last_name": "last name of each user"
    },
    "sales": {
        "id": "unique identifier for each sale",
        "user_id": "references the 'id' column in the 'users' table, linking each sale to a specific user",
        "total_amount": "the total amount associated with each sale",
        "payment_date": "the date when the payment was made",
        "sale_date": "the date when the sale was made"
    }
}

In [6]:
user_prompt = "Give me the top three users with highest amount for sales in April 2023"

application_prompt = f"""
Write a query in PostgreSQL that {user_prompt}. \
Reference the data dictionary which is in a JSON format to determine which columns and tables to use within the query: \
{data_dictionary}
"""

In [7]:
response = get_completion(application_prompt)
print(response)

SELECT u.first_name, u.last_name, SUM(s.total_amount) AS total_sales
FROM users u
JOIN sales s ON u.id = s.user_id
WHERE s.sale_date BETWEEN '2023-04-01' AND '2023-04-30'
GROUP BY u.id
ORDER BY total_sales DESC
LIMIT 3;


In [8]:
import psycopg2

try:
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        host=os.getenv('PG_HOST'),
        database=os.getenv('PG_DATABASE'),
        user=os.getenv('PG_USER'),
        password=os.getenv('PG_PASSWORD'),
        port=os.getenv('PG_PORT')
    )
    print("Connected to the PostgreSQL database successfully!")

    # Create a cursor object
    cursor = conn.cursor()
    cursor.execute("SET search_path TO coffee_chain")

    # Execute a sample query
    cursor.execute(query=response)

    # Fetch all rows
    rows = cursor.fetchall()

    print("Query results:")

    # Iterate over the rows and print the data
    for row in rows:
        print(row)
    
    cursor.close()
    conn.close()

except (Exception, psycopg2.Error) as error:
    print(f"Error while connecting to PostegreSQL: {error}")

Connected to the PostgreSQL database successfully!
Query results:
('Lauren', 'Morales', Decimal('89'))
('Manuel', 'Martin', Decimal('44'))
('Emily', 'Crawford', Decimal('39'))


In [9]:
results_prompt = f"""
The original user prompt is shown here: '{user_prompt}'.

The results from the query generated to find this data can be found here: {rows}.

Provide a user-friendly output explaining the results.
"""
response = get_completion(results_prompt)
print(response)

Here are the top three users with the highest sales amount in April 2023:

1. Lauren Morales - sold a total of $89 worth of products in April 2023.
2. Manuel Martin - sold a total of $44 worth of products in April 2023.
3. Emily Crawford - sold a total of $39 worth of products in April 2023.

These users have contributed significantly to the sales revenue in April 2023. Congratulations to them!
